In [1]:
import numpy as np
from rdkit import Chem
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import SGD
import pandas as pd
from torch_geometric.data import Data, DataLoader

In [2]:
sider = pd.read_csv("../data/sider.csv")
sider

,smiles,Hepatobiliary disorders,Metabolism and nutrition disorders,Product issues,Eye disorders,Investigations,Musculoskeletal and connective tissue disorders,Gastrointestinal disorders,Social circumstances,Immune system disorders,...,"Congenital, familial and genetic disorders",Infections and infestations,"Respiratory, thoracic and mediastinal disorders",Psychiatric disorders,Renal and urinary disorders,"Pregnancy, puerperium and perinatal conditions",Ear and labyrinth disorders,Cardiac disorders,Nervous system disorders,"Injury, poisoning and procedural complications"
0,C(CNCCNCCNCCN)N,1,1,0,0,1,1,1,0,0,...,0,0,1,1,0,0,1,1,1,0
1,CC(C)(C)C1=CC(=C(C=C1NC(=O)C2=CNC3=CC=CC=C3C2=...,0,1,0,0,1,1,1,0,0,...,0,1,1,0,0,0,1,0,1,0
2,CC[C@]12CC(=C)[C@H]3[C@H]([C@@H]1CC[C@]2(C#C)O...,0,1,0,1,1,0,1,0,1,...,0,0,0,1,0,0,0,0,1,0
3,CCC12CC(=C)C3C(C1CC[C@]2(C#C)O)CCC4=CC(=O)CCC34,1,1,0,1,1,1,1,0,1,...,1,1,1,1,1,1,0,0,1,1
4,C1C(C2=CC=CC=C2N(C3=CC=CC=C31)C(=O)N)O,1,1,0,1,1,1,1,0,1,...,0,1,1,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422,C[C@H]1CN(CC[C@@]1(C)C2=CC(=CC=C2)O)C[C@H](CC3...,0,1,0,0,0,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1423,CC[C@@H]1[C@@]2([C@@H]([C@@H](C(=O)[C@@H](C[C@...,1,1,0,1,1,1,1,0,1,...,0,1,1,1,1,0,1,1,1,1
1424,CCOC1=CC=C(C=C1)CC2=C(C=CC(=C2)[C@H]3[C@@H]([C...,1,1,0,0,1,1,1,0,1,...,0,1,0,0,1,0,0,1,1,1
1425,C1CN(CCC1N2C3=CC=CC=C3NC2=O)CCCC(C4=CC=C(C=C4)...,0,1,0,1,1,1,1,0,0,...,0,0,0,1,1,0,0,1,1,1


In [3]:
test_smi = sider["smiles"][3]
mol = Chem.MolFromSmiles(test_smi)

edges = []
for bond in mol.GetBonds():
    i = bond.GetBeginAtomIdx()
    j = bond.GetEndAtomIdx()
    edges.extend([(i,j), (j,i)])

edge_idx = list(zip(*edges))

In [4]:
def atom_feats(atom):
    return [
        atom.GetAtomicNum(),
        atom.GetDegree(),
        atom.GetNumImplicitHs(),
        atom.GetIsAromatic()
    ]

def bond_feats(bond):
    return [
        bond.GetBondType(),
        bond.GetStereo()
    ]

In [5]:
node_f = [atom_feats(a) for a in mol.GetAtoms()]
edge_f = [bond_feats(b) for b in mol.GetBonds()]

In [6]:
# Put this into a graph to use in PyG

g = Data(x=torch.Tensor(node_f),
        edge_index=torch.LongTensor(edge_idx),
        edge_attr=torch.Tensor(edge_f),
        smiles=test_smi,
        mol=mol)
g

Data(x=[24, 4], edge_index=[2, 54], edge_attr=[27, 2], smiles='CCC12CC(=C)C3C(C1CC[C@]2(C#C)O)CCC4=CC(=O)CCC34', mol=<rdkit.Chem.rdchem.Mol object at 0x7f56dccc67a0>)

In [7]:
def smi_to_PyG(smi, task_):
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return None
    edges = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edges.extend([(i,j), (j,i)])

    edge_idx = list(zip(*edges))
    node_f = [atom_feats(a) for a in mol.GetAtoms()]
    edge_f = [bond_feats(b) for b in mol.GetBonds()]

    return Data(x=torch.Tensor(node_f),
               edge_index=torch.LongTensor(edge_idx),
               edge_attr=torch.Tensor(edge_f),
               y=torch.Tensor([task_]))

In [8]:
class GetData(Dataset):
    def __init__(self, smiles, task):
        mols = [smi_to_PyG(smi, task_) for smi, task_\
               in zip(smiles, task)]
        self.X = [mol for mol in mols if mol] #If valid graph

    def __getitem__(self, idx):
        return self.X[idx]

    def __len__(self):
        return len(self.X)

In [9]:
sider_PyG = GetData(sider["smiles"], sider["Hepatobiliary disorders"])

[16:33:27] WARNING: not removing hydrogen atom without neighbors
[16:33:27] WARNING: not removing hydrogen atom without neighbors
[16:33:27] WARNING: not removing hydrogen atom without neighbors
[16:33:27] WARNING: not removing hydrogen atom without neighbors
[16:33:27] WARNING: not removing hydrogen atom without neighbors
[16:33:27] WARNING: not removing hydrogen atom without neighbors
[16:33:27] WARNING: not removing hydrogen atom without neighbors
[16:33:27] WARNING: not removing hydrogen atom without neighbors
[16:33:27] WARNING: not removing hydrogen atom without neighbors
[16:33:27] WARNING: not removing hydrogen atom without neighbors
[16:33:27] WARNING: not removing hydrogen atom without neighbors
[16:33:27] WARNING: not removing hydrogen atom without neighbors
[16:33:27] WARNING: not removing hydrogen atom without neighbors


In [10]:
len(sider_PyG)
g_test = sider_PyG[1000]
g_test

Data(x=[21, 4], edge_index=[2, 42], edge_attr=[21, 2], y=[1])

In [11]:
train_size = 0.7
val_size = 0.15
test_size = 0.15
N = len(sider_PyG)
train_loader = DataLoader(sider_PyG[:int(N * train_size)], batch_size=16, shuffle=True)
val_loader = DataLoader(sider_PyG[int(N * train_size): int(N * (train_size + val_size))], batch_size=16, shuffle=True)
test_loader = DataLoader(sider_PyG[int(N * (train_size + val_size)):], batch_size=16, shuffle=True)

/home/fibra/anaconda3/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [12]:
test = sider_PyG[1]
test.x.shape

torch.Size([29, 4])

In [13]:
val_loader.dataset[0].num_node_features

4

In [14]:
from torch.nn import functional as F
F.relu(torch.tensor(2))
F.sigmoid(torch.tensor(2))

tensor(0.8808)

In [19]:
for i, batch in enumerate(train_loader):
    print(i, batch)
    if i == 5:
        break

0 DataBatch(x=[951, 4], edge_index=[2, 1948], edge_attr=[974, 2], y=[16], batch=[951], ptr=[17])
1 DataBatch(x=[330, 4], edge_index=[2, 690], edge_attr=[345, 2], y=[16], batch=[330], ptr=[17])
2 DataBatch(x=[550, 4], edge_index=[2, 1166], edge_attr=[583, 2], y=[16], batch=[550], ptr=[17])
3 DataBatch(x=[574, 4], edge_index=[2, 1236], edge_attr=[618, 2], y=[16], batch=[574], ptr=[17])
4 DataBatch(x=[833, 4], edge_index=[2, 1718], edge_attr=[859, 2], y=[16], batch=[833], ptr=[17])
5 DataBatch(x=[345, 4], edge_index=[2, 732], edge_attr=[366, 2], y=[16], batch=[345], ptr=[17])


In [21]:
len(train_loader.dataset)

998